<a href="https://colab.research.google.com/github/robertheubanks/LLM-Engineering-Homework/blob/main/Eubanks_Week3_Assignment5_%233_Copy_of_Model_Evaluation_Instruct_tuned_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ⚠ IMPORTANT ⚠

Please ensure your Colab runtime is set to the following:

A100 GPU

Evaluation and instruction-tuning a LLM is a resource intensive process - please make sure you're using the appropriate instance.

# Evaluating an Instruct-tuned Model

Now that we've spent some time creating models with:

- Unsupervised pre-training
- Supervised fine-tuning
- Some instruction-tuning

We're ready to begin to think about how we can evaluate these models.

## Instruct-tuned Evaluation

We will now repeat the process we used on our baseline - but using the instruct-tuned version of our model!

### Load Mistral AI's Mistral-7B in 4-bit Quantization

Let's grab our dependencies, and load our model!

In [1]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 12.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

Conforming to previous notebooks - let's set up our quantization config for our model.

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

Now we have our quantization settings confirmed - let's load up our model!

In [4]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

####❓ Question:

Taking a look at the [model card](https://huggingface.co/mistralai/Mistral-7B-v0.1) (and the linked resources on the card) is this an instruct-tuned model or not?

ANSWER:
this is an instruct-tuned variant of Mistral 7B. The base model itself is designed for high performance and can be fine-tuned for specific tasks, including instruction following. The specific mention of "Mistral 7B Instruct" being fine-tuned on instruction datasets and outperforming other models in instruction-following benchmarks strongly indicates that this variant is indeed an instruct-tuned model.

### Collect and Load the Eleuther AI Evaluation Harness

Now that we have our baseline model loaded - we need to evaluate it.

For that, we'll use a tool called [Eleuther AI's LM evaluation harness](https://github.com/EleutherAI/lm-evaluation-harness). This is a specialized tool for running benchmarks on various language tasks.

Let's start by grabbing and installing it!

Why Eleuther AI's Evaluation Harness? Well - it's what powers the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)!

In [5]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
%cd lm-evaluation-harness
!pip install -e .

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 30525, done.
remote: Counting objects: 100% (1008/1008), done.
remote: Compressing objects: 100% (467/467), done.
remote: Total 30525 (delta 664), reused 828 (delta 537), pack-reused 29517
Receiving objects: 100% (30525/30525), 22.06 MiB | 12.98 MiB/s, done.
Resolving deltas: 100% (21350/21350), done.
/content/lm-evaluation-harness
Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/10

Now, we can cast our model to the desired format.

In [6]:
import lm_eval
from lm_eval.models.huggingface import HFLM
eval_model = HFLM(model, batch_size=16)

We'll set up our tasks so we can leverage them at evaluation time!

In [7]:
lm_eval.tasks.initialize_tasks()

Next, we can evaluate our base model!

>NOTE: This step will take ~30-40min. to run in full on the A100 - so ensure you set aside time to run it fully if you desire!

We're going to leverage two benchmarks today:

- [HellaSwag](https://rowanzellers.com/hellaswag/)
- [ARC Easy](https://leaderboard.allenai.org/arc_easy/submissions/get-started)
- A subset of the [MMLU benchmark](https://paperswithcode.com/dataset/mmlu), focusing only on the `college_mathematics` task.

These are lightweight benchmarks used to "score" models against eachother on the OpenLM leaderboard.

We'll consider a simple average of their scores as the "overall" score of the baseline model.

You could easily extend the number of tasks considered if you wanted to more exactly emulate the Open LLM Leaderboard.

In [8]:
results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["hellaswag", "arc_easy"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hellaswag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hellaswag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10042 [00:00<?, ? examples/s]

Map:   0%|          | 0/39905 [00:00<?, ? examples/s]

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 49669/49669 [07:22<00:00, 112.25it/s]


In [9]:
import pandas as pd

pd.DataFrame(results["results"])

,hellaswag,arc_easy
"acc,none",0.655845,0.819444
"acc_stderr,none",0.004741,0.007893
"acc_norm,none",0.833201,0.771044
"acc_norm_stderr,none",0.00372,0.008622
alias,hellaswag,arc_easy


### Few-shot MMLU (Machine Learning)

In [10]:
fs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=5,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 448/448 [00:34<00:00, 13.13it/s]


In [11]:
import pandas as pd

pd.DataFrame(fs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.482143
"acc_norm,none",0.482143
"acc_norm_stderr,none",0.047428
"acc_stderr,none",0.047428
alias,mmlu_flan_n_shot_loglikelihood_machine_learning


### Zero-Shot MMLU (Machine Learning)

In [12]:
zs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 448/448 [00:05<00:00, 85.22it/s] 


In [13]:
import pandas as pd

pd.DataFrame(zs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.482143
"acc_norm,none",0.482143
"acc_norm_stderr,none",0.047428
"acc_stderr,none",0.047428
alias,mmlu_flan_n_shot_loglikelihood_machine_learning


In [14]:
cot_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_cot_zeroshot_conceptual_physics"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

INFO:lm-eval:num_fewshot has been set to 0 for mmlu_flan_cot_zeroshot_conceptual_physics in its config. Manual configuration will be ignored.
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running generate_until requests
  0%|          | 0/26 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 26/26 [00:44<00:00,  1.71s/it]


In [15]:
import pandas as pd

pd.DataFrame(cot_mmlu_results["results"])

,mmlu_flan_cot_zeroshot_conceptual_physics
alias,mmlu_flan_cot_zeroshot_conceptual_physics
"exact_match,get-answer",0.153846
"exact_match_stderr,get-answer",0.07216


####❓Question:

What *exactly* are these two benchmarks measuring?

ANSWER:
exact_match_get-answer indicates the proportion of answers that were exactly correct.
exact_match_stderr_get-answer representS the standard error of the exact match metric, providing an indication of the variability or confidence in the exact match results.
These benchmarks are measuring the model's ability to correctly answer questions without any fine-tuning specific to the task (zero-shot), and they provide both the performance metric and its associated standard error for statistical robustness.